In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
from gensim.models import KeyedVectors
from datetime import datetime
import bert_score
from custom_score.score import score, DynamicEmbeddingSampleTest, StaticEmbeddingSampleTest 
from custom_score.utils import model_load, encode
import pickle
#import tensorflow_datasets as tfds


c:\Users\orteg\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\orteg\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\orteg\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\orteg\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## I - Custom BERTScore Build

In [1]:
ref = ["Mon prenom est marius"]
can = ["Je suis marius"]

### 1 - Token Representation

In [14]:
def model_load(model):
    assert(type(model) == str)
    if model == "Word2Vec":
        wordvector_path = r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\GoogleNews-vectors-negative300.bin.gz'
        emb = KeyedVectors.load_word2vec_format(wordvector_path, binary=True)
    if model == "Glove":
        glove_path = r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt'
        emb = KeyedVectors.load_word2vec_format(glove_path)
    else:
        print("Model not currently supported")
    return emb

In [15]:
def encode(corpus, model):
    encoded_corpus = []
    unknown = 0
    for sentence in corpus:
        encoded_sentence = []
        for word in sentence.split(" "):
            try:
                encoded_sentence.append(model[word])
            except:
                unknown += 1
        encoded_corpus.append(encoded_sentence)
    return np.array(encoded_corpus, dtype=object), unknown

##### Word2Vec

In [7]:
w2v = model_load("Word2Vec")

In [24]:
#references, n_unknown_ref = encode(["I am Marius", "I like trains"], w2v)
#candidates, n_unknown_cand = encode(["My name is Marius", "I enjoy rail vehicules"], w2v)

references, n_unknown_ref = encode(["I am Marius"], w2v)
candidates, n_unknown_cand = encode(["My name is Marius"], w2v)

In [25]:
print("Shape Reference : ", references.shape, "||", "Unknown Token Reference : ", n_unknown_ref)
print("Shape Candidate : ", candidates.shape, "||", "Unknown Token Candidate : ", n_unknown_cand)

Shape Reference :  (1, 3, 300) || Unknown Token Reference :  0
Shape Candidate :  (1, 4, 300) || Unknown Token Candidate :  0


##### Fasttext

Conversition to word2vec format (Do not compile unless necessary)

In [50]:
import fasttext

In [ ]:
ft_model = fasttext.load_model(r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\wiki.en\wiki.en.bin')
kv = KeyedVectors(vector_size=ft_model.get_dimension())
all_words = ft_model.get_words()
all_vectors = [ft_model.get_word_vector(w) for w in all_words]
kv.add_vectors(all_words, all_vectors)
kv.save_word2vec_format('ftwords.txt', binary=False)

Loading of formated fasttext file :

In [47]:
fasttext = KeyedVectors.load_word2vec_format(r'D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\ftwords.txt')

In [ ]:
fasttext = KeyedVectors.load_word2vec_format(r'')

In [48]:
fasttext["house"]

array([ 0.75779974,  0.5959443 , -0.27542254,  0.24059881, -0.5490175 ,
        0.8300641 , -0.41410735,  0.7523287 ,  0.50460875, -0.86938876],
      dtype=float32)

##### Glove

Conversition to word2vec format (Do not compile unless necessary)

In [19]:
glove_path = r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove.6B\glove.6B.300d.txt"
glove_temp = KeyedVectors.load_word2vec_format(glove_path, no_header=True)
glove_temp.save_word2vec_format(r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt")

Loading of formated glove file :

In [24]:
glove = KeyedVectors.load_word2vec_format(r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt")

### 2 - Similarity Measure

In [16]:
def SimilarityCandToRef(references, candidates):
    proximity = lambda x, y: (np.matmul(np.transpose(x), y))/(norm(x)*norm(y))

    all_proximities = []

    for candidate, reference in zip(candidates, references):
        proximities = []
        for c_word in candidate:
            sub_proximities = []
            for r_word in reference:
                sub_proximities.append(proximity(r_word, c_word))
            proximities.append(sub_proximities)
        all_proximities.append(proximities)
    return all_proximities

def SimilarityRefToCand(references, candidates):
    proximity = lambda x, y: (np.matmul(np.transpose(x), y))/(norm(x)*norm(y))

    all_proximities = []

    for candidate, reference in zip(candidates, references):
        proximities = []
        for r_word in reference:
            sub_proximities = []
            for c_word in candidate:
                sub_proximities.append(proximity(r_word, c_word))
            proximities.append(sub_proximities)
        all_proximities.append(proximities)
    return all_proximities

In [17]:
candToRef = SimilarityCandToRef(references, candidates)
refToCand = SimilarityRefToCand(references, candidates)
refToCand2 = np.transpose(candToRef) 

NameError: name 'references' is not defined

In [27]:
candToRef

[[[0.519005613856723, 0.18616442214576517, 0.04520518544899319],
  [0.09242192671658575, 0.012498354833030686, 0.025547976598766665],
  [0.2098326692090991, 0.3489852938028225, -0.00109822157119661],
  [0.0368609406903856, 0.020396980932637056, 1.0]]]

In [28]:
refToCand

[[[0.519005613856723,
   0.09242192671658575,
   0.2098326692090991,
   0.0368609406903856],
  [0.18616442214576517,
   0.012498354833030686,
   0.3489852938028225,
   0.020396980932637056],
  [0.04520518544899319, 0.025547976598766665, -0.00109822157119661, 1.0]]]

In [29]:
refToCand2

array([[[ 0.51900561],
        [ 0.09242193],
        [ 0.20983267],
        [ 0.03686094]],

       [[ 0.18616442],
        [ 0.01249835],
        [ 0.34898529],
        [ 0.02039698]],

       [[ 0.04520519],
        [ 0.02554798],
        [-0.00109822],
        [ 1.        ]]])

### 3 - Calculation of P, R and F

##### R calculation

In [44]:
fullSum = []
for individualSimilarity in candToRef:
    currentSum = 0
    for row in individualSimilarity:
        currentSum += row[np.argmax(row)]
    fullSum.append(currentSum)
R = []
for sum, reference in zip(fullSum, references):
    R.append((1/norm(reference))*sum)


In [45]:
R

[0.42425704559238325]

##### P Calculation

In [46]:
fullSum = []
for individualSimilarity in refToCand:
    currentSum = 0
    for row in individualSimilarity:
        currentSum += row[np.argmax(row)]
    fullSum.append(currentSum)
P = []
for sum, candidate in zip(fullSum, candidates):
    P.append((1/norm(candidate))*sum)

In [47]:
P

[0.3604158590807159]

##### F Calculation

In [48]:
F = []

for r, p in zip(R, P):
    f = 2*((p*r)/(p+r))
    F.append(f)

In [49]:
F

[0.38973938477441966]

##### Full Calculation

In [ ]:
def computeMetrics(candToRef, refToCand):
    # R computation
    fullSum = []
    for individualSimilarity in candToRef:
        currentSum = 0
        for row in individualSimilarity:
            currentSum += row[np.argmax(row)]
        fullSum.append(currentSum)
    R = []
    for sum, reference in zip(fullSum, references):
        R.append((1/norm(reference))*sum)

    # P computation
    fullSum = []
    for individualSimilarity in refToCand:
        currentSum = 0
        for row in individualSimilarity:
            currentSum += row[np.argmax(row)]
        fullSum.append(currentSum)
    P = []
    for sum, candidate in zip(fullSum, candidates):
        P.append((1/norm(candidate))*sum)
    
    # F computation
    F = []
    for r, p in zip(R, P):
        f = 2*((p*r)/(p+r))
        F.append(f)
    
    return (R, R, F)

### 4 - IDF Weighting

In [3]:
from custom_score.utils import computeIdf
from custom_score.utils import getIdf

In [4]:
idfDict = computeIdf(ref[0])

In [6]:
getIdf(idfDict, "coucou")

1.0

### 5 - Simplify Function

##### Function creation and testing

In [10]:
from random import uniform

In [14]:
def cleanString(string, maxSpacing=10):
    """
    :param1 string (string): Initial corpus
    :param2 maxSpacing (int): Maximal number of adjacent space to be found and suppressed in the corpus.

    :output clean (string): Cleansed corpus
    """

    clean = string[:]

    #remove linebreaks  
    clean = clean.replace("\n", " ")
    clean = clean.replace("-", "")

    #remove surplus spacing
    spacing = "".join([" " for _ in range(maxSpacing)])
    for _ in range(maxSpacing-1):
        spacing = spacing[:-1]
        clean = clean.replace(spacing, " ")
    
    return clean

def sentenceSelection(corpus, scores, reductionFactor=2):
    """
    Returns a list of selected indices of sentence that will constituate the new corpus.

    :param1 corpus (list): List of sentences of the reference document.
    :param2 scores (list): List of the similarity scores of each sentence of the reference compared to the entire reference document.
    :param3 reductionFactor (float or int): Number determining how much the reference text will be shortened. 

    :output selected_indexes (list): List of indexes of the initial corpus sentences that have been selected.
    """
    totalLength = len(corpus)
    targetLength = int(totalLength/reductionFactor)
    selected_indexes = []
    
    randomized_scores = [np.mean([curScore, uniform(0, 1)]) for curScore in scores]
    ranking = np.argsort(randomized_scores)[::-1]
    selected_indexes = ranking[:targetLength]

    return selected_indexes

def simplify(reference, model, reductionFactor=2, maxSpacing=10):
    """
    Return a reduced string computed using static embedding vectors similarity. Also denoises the data by removing superfluous elements such as "\n" or useless signs.
 
    :param1 reference (string): Document to simplify.
    :param2 model (dict): Dictionnary of keyed-vectors.
    :param3 reductionFactor (float or int): Number determining how much the reference text will be shortened. 
    :param4 maxSpacing (int): Maximal number of adjacent space to be found and suppressed in the corpus.

    :output simplified (string): Simplified version of the initial document.
    """

    #preprocess corpus
    clean = cleanString(reference, maxSpacing)
    sentences = clean.split(".")
    sentences.pop()
    respaced_sentences = []
    for sentence in sentences:
        if sentence[0] == " ":
            sentence = sentence[1:]
        respaced_sentences.append(sentence)
    
    corpus = " ".join(respaced_sentences)
    scores = []
    for sentence in respaced_sentences:
        (R, _, _) = score(model, [sentence], [corpus])
        scores.append(R[0])

    indices = sentenceSelection(respaced_sentences, scores, reductionFactor)
    
    simplified = []
    for index in indices:
        simplified.append(respaced_sentences[index])
    
    simplified = " ".join(simplified)

    return simplified

In [11]:
test = "Je suis marius. J'aime les fruits."

In [4]:
w2v = model_load("Word2Vec", True)

In [15]:
simplify(test, w2v)

"J'aime les fruits"

In [16]:
billsum0 = "SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``National Science Education Tax \nIncentive for Businesses Act of 2007''.\n\nSEC. 2. CREDITS FOR CERTAIN CONTRIBUTIONS BENEFITING SCIENCE, \n              TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE \n              ELEMENTARY AND SECONDARY SCHOOL LEVEL.\n\n    (a) In General.--Subpart D of part IV of subchapter A of chapter 1 \nof the Internal Revenue Code of 1986 (relating to business related \ncredits) is amended by adding at the end the following new section:\n\n``SEC. 45O. CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, \n              AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY \n              SCHOOL LEVEL.\n\n    ``(a) In General.--For purposes of section 38, the elementary and \nsecondary science, technology, engineering, and mathematics (STEM) \ncontributions credit determined under this section for the taxable year \nis an amount equal to 100 percent of the qualified STEM contributions \nof the taxpayer for such taxable year.\n    ``(b) Qualified STEM Contributions.--For purposes of this section, \nthe term `qualified STEM contributions' means--\n            ``(1) STEM school contributions,\n            ``(2) STEM teacher externship expenses, and\n            ``(3) STEM teacher training expenses.\n    ``(c) STEM School Contributions.--For purposes of this section--\n            ``(1) In general.--The term `STEM school contributions' \n        means--\n                    ``(A) STEM property contributions, and\n                    ``(B) STEM service contributions.\n            ``(2) STEM property contributions.--The term `STEM property \n        contributions' means the amount which would (but for subsection \n        (f)) be allowed as a deduction under section 170 for a \n        charitable contribution of STEM inventory property if--\n                    ``(A) the donee is an elementary or secondary \n                school described in section 170(b)(1)(A)(ii),\n                    ``(B) substantially all of the use of the property \n                by the donee is within the United States or within the \n                defense dependents' education system for educational \n                purposes in any of the grades K-12 that are related to \n                the purpose or function of the donee,\n                    ``(C) the original use of the property begins with \n                the donee,\n                    ``(D) the property will fit productively into the \n                donee's education plan,\n                    ``(E) the property is not transferred by the donee \n                in exchange for money, other property, or services, \n                except for shipping, installation and transfer costs, \n                and\n                    ``(F) the donee's use and disposition of the \n                property will be in accordance with the provisions of \n                subparagraphs (B) and (E).\n        The determination of the amount of deduction under section 170 \n        for purposes of this paragraph shall be made as if the \n        limitation under section 170(e)(3)(B) applied to all STEM \n        inventory property.\n            ``(3) STEM service contributions.--The term `STEM service \n        contributions' means the amount paid or incurred during the \n        taxable year for STEM services provided in the United States or \n        in the defense dependents' education system for the exclusive \n        benefit of students at an elementary or secondary school \n        described in section 170(b)(1)(A)(ii) but only if--\n                    ``(A) the taxpayer is engaged in the trade or \n                business of providing such services on a commercial \n                basis, and\n                    ``(B) no charge is imposed for providing such \n                services.\n            ``(4) STEM inventory property.--The term `STEM inventory \n        property' means, with respect to any contribution to a school, \n        any property--\n                    ``(A) which is described in paragraph (1) or (2) of \n                section 1221(a) with respect to the donor, and\n                    ``(B) which is determined by the school to be \n                needed by the school in providing education in grades \n                K-12 in the areas of science, technology, engineering, \n                or mathematics.\n            ``(5) STEM services.--The term `STEM services' means, with \n        respect to any contribution to a school, any service determined \n        by the school to be needed by the school in providing education \n        in grades K-12 in the areas of science, technology, \n        engineering, or mathematics, including teaching courses of \n        instruction at such school in any such area.\n            ``(6) Defense dependents' education system.--For purposes \n        of this subsection, the term `defense dependents' education \n        system' means the program established and operated under the \n        Defense Dependents' Education Act of 1978 (20 U.S.C. 921 et \n        seq.).\n    ``(d) STEM Teacher Externship Expenses.--For purposes of this \nsection--\n            ``(1) In general.--The term `STEM teacher externship \n        expenses' means any amount paid or incurred to carry out a STEM \n        externship program of the taxpayer but only to the extent that \n        such amount is attributable to the participation in such \n        program of any eligible STEM teacher, including amounts paid to \n        such a teacher as a stipend while participating in such \n        program.\n            ``(2) STEM externship program.--The term `STEM externship \n        program' means any program--\n                    ``(A) established by a taxpayer engaged in a trade \n                or business within an area of science, technology, \n                engineering, or mathematics, and\n                    ``(B) under which eligible STEM teachers receive \n                training to enhance their teaching skills in the areas \n                of science, technology, engineering, or mathematics or \n                otherwise improve their knowledge in such areas.\n            ``(3) Eligible stem teacher.--The term `eligible STEM \n        teacher' means any individual--\n                    ``(A) who is a teacher in grades K-12 at an \n                educational organization described in section \n                170(b)(1)(A)(ii) which is located in the United States \n                or which is located on a United States military base \n                outside the United States, and\n                    ``(B) whose teaching responsibilities at such \n                school include, or are likely to include, any course in \n                the areas of science, technology, engineering, or \n                mathematics.\n    ``(e) STEM Teacher Training Expenses.--The term `STEM teacher \ntraining expenses' means any amount paid or incurred by a taxpayer \nengaged in a trade or business within an area of science, technology, \nengineering, or mathematics which is attributable to the participation \nof any eligible STEM teacher in a regular training program provided to \nemployees of the taxpayer which is determined by such teacher's school \nas enhancing such teacher's teaching skills in the areas of science, \ntechnology, engineering, or mathematics.\n    ``(f) Denial of Double Benefit.--No deduction shall be allowed \nunder this chapter for any amount allowed as a credit under this \nsection.''.\n    (b) Conforming Amendments.--\n            (1) Section 38(b) of such Code is amended by striking \n        ``plus'' at the end of paragraph (30), by striking the period \n        at the end of paragraph (31), and inserting ``, plus'', and by \n        adding at the end the following new paragraph:\n            ``(32) the elementary and secondary science, technology, \n        engineering, and mathematics (STEM) contributions credit \n        determined under section 45O.''.\n            (2) The table of sections for subpart D of part IV of \n        subchapter A of chapter 1 of such Code is amended by adding at \n        the end the following new item:\n\n``Sec. 45O. Contributions benefiting science, technology, engineering, \n                            and mathematics education at the elementary \n                            and secondary school level.''.\n    (c) Effective Date.--The amendments made by this section shall \napply to taxable years beginning after the date of the enactment of \nthis Act."

In [17]:
res = simplify(billsum0, w2v, reductionFactor=4, maxSpacing=15)
res

"The term `STEM inventory property' means, with respect to any contribution to a school, any property ``(A) which is described in paragraph (1) or (2) of section 1221(a) with respect to the donor, and ``(B) which is determined by the school to be needed by the school in providing education in grades K12 in the areas of science, technology, engineering, or mathematics The term `STEM externship program' means any program ``(A) established by a taxpayer engaged in a trade or business within an area of science, technology, engineering, or mathematics, and ``(B) under which eligible STEM teachers receive training to enhance their teaching skills in the areas of science, technology, engineering, or mathematics or otherwise improve their knowledge in such areas Contributions benefiting science, technology, engineering, and mathematics education at the elementary and secondary school level The determination of the amount of deduction under section 170 for purposes of this paragraph shall be ma

##### Package testing

In [3]:
from custom_score.score import simplify

In [4]:
billsum0 = "SECTION 1. SHORT TITLE.\n\n    This Act may be cited as the ``National Science Education Tax \nIncentive for Businesses Act of 2007''.\n\nSEC. 2. CREDITS FOR CERTAIN CONTRIBUTIONS BENEFITING SCIENCE, \n              TECHNOLOGY, ENGINEERING, AND MATHEMATICS EDUCATION AT THE \n              ELEMENTARY AND SECONDARY SCHOOL LEVEL.\n\n    (a) In General.--Subpart D of part IV of subchapter A of chapter 1 \nof the Internal Revenue Code of 1986 (relating to business related \ncredits) is amended by adding at the end the following new section:\n\n``SEC. 45O. CONTRIBUTIONS BENEFITING SCIENCE, TECHNOLOGY, ENGINEERING, \n              AND MATHEMATICS EDUCATION AT THE ELEMENTARY AND SECONDARY \n              SCHOOL LEVEL.\n\n    ``(a) In General.--For purposes of section 38, the elementary and \nsecondary science, technology, engineering, and mathematics (STEM) \ncontributions credit determined under this section for the taxable year \nis an amount equal to 100 percent of the qualified STEM contributions \nof the taxpayer for such taxable year.\n    ``(b) Qualified STEM Contributions.--For purposes of this section, \nthe term `qualified STEM contributions' means--\n            ``(1) STEM school contributions,\n            ``(2) STEM teacher externship expenses, and\n            ``(3) STEM teacher training expenses.\n    ``(c) STEM School Contributions.--For purposes of this section--\n            ``(1) In general.--The term `STEM school contributions' \n        means--\n                    ``(A) STEM property contributions, and\n                    ``(B) STEM service contributions.\n            ``(2) STEM property contributions.--The term `STEM property \n        contributions' means the amount which would (but for subsection \n        (f)) be allowed as a deduction under section 170 for a \n        charitable contribution of STEM inventory property if--\n                    ``(A) the donee is an elementary or secondary \n                school described in section 170(b)(1)(A)(ii),\n                    ``(B) substantially all of the use of the property \n                by the donee is within the United States or within the \n                defense dependents' education system for educational \n                purposes in any of the grades K-12 that are related to \n                the purpose or function of the donee,\n                    ``(C) the original use of the property begins with \n                the donee,\n                    ``(D) the property will fit productively into the \n                donee's education plan,\n                    ``(E) the property is not transferred by the donee \n                in exchange for money, other property, or services, \n                except for shipping, installation and transfer costs, \n                and\n                    ``(F) the donee's use and disposition of the \n                property will be in accordance with the provisions of \n                subparagraphs (B) and (E).\n        The determination of the amount of deduction under section 170 \n        for purposes of this paragraph shall be made as if the \n        limitation under section 170(e)(3)(B) applied to all STEM \n        inventory property.\n            ``(3) STEM service contributions.--The term `STEM service \n        contributions' means the amount paid or incurred during the \n        taxable year for STEM services provided in the United States or \n        in the defense dependents' education system for the exclusive \n        benefit of students at an elementary or secondary school \n        described in section 170(b)(1)(A)(ii) but only if--\n                    ``(A) the taxpayer is engaged in the trade or \n                business of providing such services on a commercial \n                basis, and\n                    ``(B) no charge is imposed for providing such \n                services.\n            ``(4) STEM inventory property.--The term `STEM inventory \n        property' means, with respect to any contribution to a school, \n        any property--\n                    ``(A) which is described in paragraph (1) or (2) of \n                section 1221(a) with respect to the donor, and\n                    ``(B) which is determined by the school to be \n                needed by the school in providing education in grades \n                K-12 in the areas of science, technology, engineering, \n                or mathematics.\n            ``(5) STEM services.--The term `STEM services' means, with \n        respect to any contribution to a school, any service determined \n        by the school to be needed by the school in providing education \n        in grades K-12 in the areas of science, technology, \n        engineering, or mathematics, including teaching courses of \n        instruction at such school in any such area.\n            ``(6) Defense dependents' education system.--For purposes \n        of this subsection, the term `defense dependents' education \n        system' means the program established and operated under the \n        Defense Dependents' Education Act of 1978 (20 U.S.C. 921 et \n        seq.).\n    ``(d) STEM Teacher Externship Expenses.--For purposes of this \nsection--\n            ``(1) In general.--The term `STEM teacher externship \n        expenses' means any amount paid or incurred to carry out a STEM \n        externship program of the taxpayer but only to the extent that \n        such amount is attributable to the participation in such \n        program of any eligible STEM teacher, including amounts paid to \n        such a teacher as a stipend while participating in such \n        program.\n            ``(2) STEM externship program.--The term `STEM externship \n        program' means any program--\n                    ``(A) established by a taxpayer engaged in a trade \n                or business within an area of science, technology, \n                engineering, or mathematics, and\n                    ``(B) under which eligible STEM teachers receive \n                training to enhance their teaching skills in the areas \n                of science, technology, engineering, or mathematics or \n                otherwise improve their knowledge in such areas.\n            ``(3) Eligible stem teacher.--The term `eligible STEM \n        teacher' means any individual--\n                    ``(A) who is a teacher in grades K-12 at an \n                educational organization described in section \n                170(b)(1)(A)(ii) which is located in the United States \n                or which is located on a United States military base \n                outside the United States, and\n                    ``(B) whose teaching responsibilities at such \n                school include, or are likely to include, any course in \n                the areas of science, technology, engineering, or \n                mathematics.\n    ``(e) STEM Teacher Training Expenses.--The term `STEM teacher \ntraining expenses' means any amount paid or incurred by a taxpayer \nengaged in a trade or business within an area of science, technology, \nengineering, or mathematics which is attributable to the participation \nof any eligible STEM teacher in a regular training program provided to \nemployees of the taxpayer which is determined by such teacher's school \nas enhancing such teacher's teaching skills in the areas of science, \ntechnology, engineering, or mathematics.\n    ``(f) Denial of Double Benefit.--No deduction shall be allowed \nunder this chapter for any amount allowed as a credit under this \nsection.''.\n    (b) Conforming Amendments.--\n            (1) Section 38(b) of such Code is amended by striking \n        ``plus'' at the end of paragraph (30), by striking the period \n        at the end of paragraph (31), and inserting ``, plus'', and by \n        adding at the end the following new paragraph:\n            ``(32) the elementary and secondary science, technology, \n        engineering, and mathematics (STEM) contributions credit \n        determined under section 45O.''.\n            (2) The table of sections for subpart D of part IV of \n        subchapter A of chapter 1 of such Code is amended by adding at \n        the end the following new item:\n\n``Sec. 45O. Contributions benefiting science, technology, engineering, \n                            and mathematics education at the elementary \n                            and secondary school level.''.\n    (c) Effective Date.--The amendments made by this section shall \napply to taxable years beginning after the date of the enactment of \nthis Act."

In [ ]:
simplify(billsum0, w2v, 4, 15)

### 6 - Basic Test of the package

In [3]:
from custom_score.score import score
from custom_score.utils import model_load

In [2]:
w2v = model_load("Word2Vec")

In [5]:
score(w2v)

([0.5415885295887227], [0.6913107305461017], [0.6073585639177453])

In [29]:
score(glove)

([0.03954665010154433], [0.05171590963278553], [0.04481993467824077])

In [4]:
score(w2v, withIdf=True)

([0.6951478044587418], [0.6913107305461019], [0.6932239578823217])

## II - Billsum Dataset Test

### 0 - Utils

In [2]:
def DynamicEmbeddingSampleTest(data, limit=3, modelPath = None, model = None, nbLayers = 24):
    nbIter = 1
    scores = []
    init_time = datetime.now()
    for row in data.iterrows():
        curCand = [row[1][2]]
        curRef = row[1][1]
        curCand = [" ".join(row[1][2].split("\n"))]
        curRef = [" ".join(row[1][1].split("\n"))]
        assert len(curCand) == len(curRef)
        if modelPath != None:
            (P, R, F), hashname = bert_score.score(curCand, curRef, lang="en", 
                                        model_type=modelPath, 
                                        num_layers=nbLayers, return_hash=True)
        elif model != None:
            (P, R, F), hashname = bert_score.score(curCand, curRef, lang="en", 
                                        model_type=model, 
                                        num_layers=nbLayers, return_hash=True)
        else:
            (P, R, F), hashname = bert_score.score(curCand, curRef, lang="en", return_hash=True)
        P = P[0].item()
        R = R[0].item()
        F = F[0].item()
        scores.append((P, R, F))
        if nbIter >= limit:
            break
        nbIter += 1
    runtime = (datetime.now() - init_time).total_seconds()
    return scores, runtime

In [3]:
def StaticEmbeddingSampleTest(data, model, limit=3):
    nbIter = 1
    scores = []
    init_time = datetime.now()
    for row in data.iterrows():
        curCand = [row[1][2]]
        curRef = row[1][1]
        curCand = [" ".join(row[1][2].split("\n"))]
        curRef = [" ".join(row[1][1].split("\n"))]
        assert len(curCand) == len(curRef)
        
        (P, R, F) = score(model, curCand, curRef)
        P = P[0]
        R = R[0]
        F = F[0]
        scores.append((P, R, F))

        if nbIter >= limit:
            break
        nbIter += 1
    runtime = (datetime.now() - init_time).total_seconds()
    return scores, runtime
    

### 1 - Load Benchmarking Dataset

##### Load Locally

In [16]:
billsumPath = r"D:\COURS\A4\S8\Stage\Documents\Datasets\Summary Evaluation\BillSum\corpus"

In [17]:
#billsum_train = pd.read_json(path_or_buf = billsumPath + r"\us_train_data_final_OFFICIAL.jsonl", lines=True)
billsum_test = pd.read_json(path_or_buf = billsumPath + r"\us_test_data_final_OFFICIAL.jsonl", lines=True)
billsum_test = billsum_test.loc[:, ["text", "summary"]]
billsum_test.head(3)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...


##### Load Remotly

In [2]:
billsum = tfds.load('huggingface:billsum')

In [3]:
billsum_test = tfds.as_dataframe(billsum["test"])
billsum_test = billsum_test.loc[:, ["text", "summary"]]
billsum_test.text = billsum_test.text.str.decode("utf-8")
billsum_test.summary = billsum_test.summary.str.decode("utf-8")
billsum_test.head(5)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Local Innovation and Coastal Protection Act of...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Gun Show Background Check Act of 2008 - Amends...
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,Recycled Roads Act of 2003 - Directs the Secre...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,Prosthetic and Custom Orthotic Parity Act of 2...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,"Investing in Neighborhood-focused, Vital, Evid..."


### 2 - Test : RoBERTa Implementation

In [4]:
bert_scores, bert_runtime = DynamicEmbeddingSampleTest(billsum_test)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaM

### 3 - Test : Word2Vec Implementation 

In [4]:
path_w2v_serialized = r"D:\COURS\A4\S8 - ESILV\Stage\Work\Models\serialized_w2v.pkl"

##### W2V Serialization

In [2]:
w2v = model_load("Word2Vec")

In [9]:
with open(path_w2v_serialized, 'wb') as f:
    pickle.dump(w2v, f)
    f.close()

##### W2V test

In [5]:
with open(path_w2v_serialized, 'rb') as f:
    w2v = pickle.load(f)
    f.close()

In [9]:
word2vec_scores, word2vec_runtime = StaticEmbeddingSampleTest(billsum_test, w2v, 3, withIdf = False)

### 4 - Test : Fasttext Implementation 

### 5 - Test : Glove Implementation

In [9]:
glove = KeyedVectors.load_word2vec_format(r"D:\COURS\A4\S8\Stage\Documents\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\custom_BERTScore\glove2word2vec.txt")

In [10]:
glove_scores, glove_runtime = StaticEmbeddingSampleTest(billsum_test, glove)

### 6 - Evaluation

##### Runtime Evaluation

In [12]:
runtimeTable = [bert_runtime, word2vec_runtime, glove_runtime]
runtimeDf = pd.DataFrame(runtimeTable, columns=["runtime"], index=["Roberta-24-layers", "Word2Vec", "Glove"])
runtimeDf

,runtime
Roberta-24-layers,38.338537
Word2Vec,4.411479
Glove,4.890593


##### Quality Evaluation

In [44]:
bert_scores_Df = pd.DataFrame(bert_scores, columns=["Bert_P", "Bert_R", "Bert_F"])
bert_scores_Df

,Bert_P,Bert_R,Bert_F
0,0.874369,0.704338,0.780197
1,0.846002,0.728316,0.782760
2,0.873844,0.702725,0.778998


In [10]:
word2vec_scores_Df = pd.DataFrame(word2vec_scores, columns=["W2V_P", "W2V_R", "W2V_F"])
word2vec_scores_Df

,W2V_P,W2V_R,W2V_F
0,0.646725,0.902619,0.753539
1,0.699015,0.941868,0.802470
2,0.652091,0.959232,0.776389


In [8]:
word2vec_scores_Df = pd.DataFrame(word2vec_scores, columns=["W2V_P_IDF", "W2V_R_IDF", "W2V_F_IDF"])
word2vec_scores_Df

,W2V_P_IDF,W2V_R_IDF,W2V_F_IDF
0,0.636382,0.907588,0.748166
1,0.623577,0.943714,0.750950
2,0.593763,0.958194,0.733191


In [20]:
billsum_test.head(10)

,bill_id,text,summary,title,text_len,sum_len
0,110_hr37,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...,To amend the Internal Revenue Code of 1986 to ...,8494,321
1,112_hr2873,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...,To amend the Internal Revenue Code of 1986 to ...,6522,1424
2,109_s2408,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...,A bill to require the Director of National Int...,6154,463
3,108_s1899,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...,A bill to improve data collection and dissemin...,19853,1400
4,107_s1531,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...,A bill to amend the Internal Revenue Code of 1...,6273,278
5,107_hr4541,SECTION 1. RELIQUIDATION OF CERTAIN ENTRIES PR...,Requires the Customs Service to reliquidate ce...,To provide for reliquidation of entries premat...,11691,114
6,111_s1495,SECTION 1. SHORT TITLE.\n\n This Act may be...,Service Dogs for Veterans Act of 2009 - Direct...,A bill to require the Secretary of Veterans Af...,5328,379
7,111_s3885,SECTION 1. SHORT TITLE.\n\n This Act may be...,Race to the Top Act of 2010 - Directs the Secr...,A bill to provide incentives for States and lo...,16668,1525
8,113_hr1796,SECTION 1. SHORT TITLE.\n\n This Act may be...,Troop Talent Act of 2013 - Directs the Secreta...,Troop Talent Act of 2013,15352,2151
9,103_hr1987,SECTION 1. SHORT TITLE.\n\n This Act may be...,Taxpayer's Right to View Act of 1993 - Amends ...,Taxpayer's Right to View Act of 1993,5633,894


In [12]:
glove_scores_Df = pd.DataFrame(glove_scores, columns=["GLV_P", "GLV_R", "GLV_F"])
glove_scores_Df

,GLV_P,GLV_R,GLV_F
0,0.193759,-0.170188,-2.798021
1,1.496544,-0.422760,-1.178411
2,0.309721,-0.217137,-1.452770


# HOLDALL

### Padding

In [ ]:
# Padding
new_references = []
new_candidates = []

for reference, candidate in zip(references, candidates):
    size_diff = len(reference) - len(candidate)
    if size_diff >= 0:
        candidate = np.pad(candidate, (0, size_diff))
        reference = np.array(reference)
    else:
        reference = np.pad(reference, [(0, np.abs(size_diff)), (0, 0)], mode="constant")
        candidate = np.array(candidate)
    new_references.append(reference)
    new_candidates.append(candidate)
   

references = np.array(new_references, dtype=object)
candidates = np.array(new_candidates, dtype=object)

### Duplicating billsum summary column

In [ ]:
tab = []
for i in range(billsum_test.shape[0]):
    tab.append( billsum_test.iloc[i][2] + billsum_test.iloc[i][2])
billsum_test["summary"] = tab    

### Auto cosine similarities

In [ ]:
def autoSimilarity(sentences):
    """
    :param1 sentences (list): List of sentences of the reference corpus.

    :output similarities (list): List of cosine similarities between each sentence and the 
    """
    proximity = lambda x, y: (np.matmul(np.transpose(x), y))/(norm(x)*norm(y))
    all = " ".join(sentences)
    similarities = []
    for sentence in sentences:
        currentSimilarity = []
        for word in sentence.split(" "):
            similarities.append(proximity(sentences, sentence))
    return similarities

### Legacy version : sentence ranking

In [ ]:
def sentenceSelection(corpus, scores, reductionFactor=2):
    """
    Returns a list of selected indices of sentence that will constituate the new corpus.

    :param1 corpus (list): List of sentences of the reference document.
    :param2 scores (list): List of the similarity scores of each sentence of the reference compared to the entire reference document.
    :param3 reductionFactor (float or int): Number determining how much the reference text will be shortened. 

    :output selected_indexes (list): List of indexes of the initial corpus sentences that have been selected.
    """
    totalLength = len(corpus)
    targetLength = int(totalLength/reductionFactor)
    selected_indexes = []
    for _ in range(targetLength):
        randomized_scores = [np.mean([curScore, uniform(0, 1)]) for curScore in scores]
        ranking = np.argsort(randomized_scores)[::-1]
        for index in ranking:
            if not index in selected_indexes:
                selected_indexes.append(index)
                break
    return selected_indexes